Importar Bibliotecas:

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from google.cloud import bigquery
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Configurar cliente BigQuery
client = bigquery.Client()

# Consulta SQL para carregar os dados
query = """
SELECT vote_average, release_date, vote_count, popularity
FROM `gcp-pde-344913.proj_estudo.movies`
"""

# Executar a consulta
query_job = client.query(query)

# Transformar o resultado em um DataFrame pandas
data = query_job.to_dataframe()

# Pré-processamento dos dados
data['release_year'] = pd.to_datetime(data['release_date']).dt.year  # Extraindo o ano de lançamento
data.drop(columns=['release_date'], inplace=True)  # Removendo a coluna original de data
data.dropna(inplace=True)  # Remover linhas com valores nulos

# Verificar se ainda há dados após a remoção de linhas nulas
if data.shape[0] == 0:
    raise ValueError("Todos os dados foram removidos devido a valores nulos. Não é possível continuar.")

# Separar x e y
x = data.drop(columns=['vote_average'])  # Features
y = data['vote_average']  # Target

# Separar treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Identificar colunas categóricas
categorical_columns = x.select_dtypes(exclude=np.number).columns

# Criar transformador para colunas numéricas
num_transf = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# Criar transformador para colunas categóricas
cat_transf = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar transformadores para todas as colunas
preprocessor = ColumnTransformer([
    ('num', num_transf, x.select_dtypes(include=np.number).columns),
    ('cat', cat_transf, categorical_columns)
])

# Configurar o pipeline do modelo
svm = Pipeline([
    ('preprocessor', preprocessor),
    ('model', SVR())
])

# Configurar os parâmetros de busca
params_svr = {
    'model__kernel': ['rbf', 'linear', 'poly'],
    'model__C': np.logspace(-3, 2, 6),
    'model__gamma': np.logspace(-3, 2, 6),
    'model__degree': [2, 3, 4]
}

# Configurar o buscador
random_search_svr = RandomizedSearchCV(
    estimator=svm,
    param_distributions=params_svr,
    n_iter=50,
    scoring='neg_mean_squared_error',  # Usando o erro quadrático médio negativo como métrica para regressão
    cv=5,
    refit=True,
    error_score=0
)

# Ajustar o modelo
random_search_svr.fit(x_train, y_train)

# Melhores parâmetros
print('SVR:', random_search_svr.best_params_)

# Melhor desempenho médio
print('SVR - Melhor MSE:', -random_search_svr.best_score_)

# Avaliar o modelo
y_pred_train = random_search_svr.predict(x_train)
y_pred_test = random_search_svr.predict(x_test)

# Avaliar o desempenho
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE (treino):', mse_train)
print('MSE (teste):', mse_test)

# Exportar os resultados para um arquivo CSV
results_df = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred_test})
results_df.to_csv('gs://us-central1-composer-proj-e-a59bc4b9-bucket/CSV/results.csv', index=False)


SVR: {'model__kernel': 'poly', 'model__gamma': 100.0, 'model__degree': 4, 'model__C': 0.1}
SVR - Melhor MSE: 0.0015132166666666658
MSE (treino): 0.0014733749999999975
MSE (teste): 0.01872775000000003
